In [41]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import * 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from urllib.request import urlretrieve
from owslib.wfs import WebFeatureService
from dotenv import load_dotenv
from pyspark.sql import functions as F
from pyspark.sql import Window
from pyspark.sql.functions import lit

# Create a spark session
spark = (
    SparkSession.builder.appName("analysis")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "2g")
    .config("spark.executor.memory", "6g")
    .getOrCreate()
)

## Initial Ranking System

Ranking System uses the following features:
- future predictions for transactions per merchant per month
- future predictions for profit per merchant per month
- future predictions for customers within revenue range \\$50k-\\$75k per month

In [42]:
future_trans_sdf = spark.read.parquet("../data/curated/futuretrans.parquet/")
future_trans_sdf.limit(5)

year,month,merchant_abn,total_transactions
2023,5,80121625734,36.85885854384651
2023,5,10187291046,43.94976345056075
2023,5,32772607667,56.982676773785485
2023,5,19633197919,44.95620568112244
2023,5,18737319630,38.9600224762402


In [43]:
future_profit_sdf = spark.read.parquet("../data/curated/futureprofit.parquet/")
future_profit_sdf.limit(5)

year,month,merchant_abn,total_profit
2023,5,80121625734,718.7299519941164
2023,5,10187291046,321.20928697548646
2023,5,32772607667,382.856836584368
2023,5,19633197919,909.3521069894532
2023,5,18737319630,368.7870030808791


In [44]:
future_mcustomers_sdf = spark.read.parquet("../data/curated/futuremcustomers.parquet/")
future_mcustomers_sdf.limit(5)

year,month,merchant_abn,total_middle_customers
2023,4,67610254313,8.911459530226823
2023,4,76241460849,6.598423723172958
2023,4,86201937910,6.298368461310957
2023,4,53126107676,98.7534217787489
2023,4,94547309480,14.325967064940135


In [45]:
merch_pop = spark.read.parquet("../data/curated/merch_pop.parquet/")
merch_pop.limit(5)

merchant_abn,total_pop,under10_pop,adolsc_pop,yng_adult_pop,mid_age_pop,old_pop
10255988167,2,2,3,2,2,4
10430380319,3,3,3,3,3,3
10618089367,4,4,4,4,4,4
11215815177,1,1,1,1,1,1
11237511112,4,4,4,4,4,4


In [46]:
merchants_pd = future_trans_sdf \
    .select('merchant_abn') \
    .distinct() \
    .withColumn('points', lit(1)) \
    .orderBy("merchant_abn") \
    .toPandas()

merchants_pd

,merchant_abn,points
0,10023283211,1
1,10142254217,1
2,10165489824,1
3,10187291046,1
4,10192359162,1
...,...,...
4013,99938978285,1
4014,99974311662,1
4015,99976658299,1
4016,99987905597,1


Have 4018 distinct merchants, need to select top 100 merchants

Idea:
- all merchants have initial points 1
- the more the number of points, the higher/better the rank will be
- points given based on sorted positioning within the features and importance of the feature (for transaction, profit, and mcustomers)

In [47]:
WEIGHT_TRANS = 0.375
WEIGHT_PROFIT = 0.375
WEIGHT_MCUSTOMERS = 0.125
WEIGHT_POP = 0.125

***

Award 0.125 points to merchants having a relatively large young adult and middle age population (external research) <br>
with large meaning the populations are above the lower quartile (not group 1 - see notebook 4.4)

In [48]:
w = Window.partitionBy('month')

merch_pop_mod = merch_pop \
    .drop(*("total_pop", "under10_pop", "adolsc_pop", "old_pop")) \
    .withColumn(
        "points",
        when(
            (col("yng_adult_pop") != 1) &
            (col("mid_age_pop") != 1),
            lit(0.125)
        ).otherwise(lit(0))
    ) \
    .drop(*("yng_adult_pop", "mid_age_pop")) \
    .orderBy("merchant_abn")

merch_pop_mod.limit(10)

merchant_abn,points
10023283211,0.0
10142254217,0.0
10165489824,0.125
10187291046,0.125
10192359162,0.125
10206519221,0.0
10255988167,0.125
10264435225,0.0
10279061213,0.0
10323485998,0.125


In [49]:
merch_pop_mod_df = merch_pop_mod.toPandas()
merch_pop_mod_df

,merchant_abn,points
0,10023283211,0.000
1,10142254217,0.000
2,10165489824,0.125
3,10187291046,0.125
4,10192359162,0.125
...,...,...
4013,99938978285,0.000
4014,99974311662,0.000
4015,99976658299,0.125
4016,99987905597,0.125


In [50]:
merchants_pd

,merchant_abn,points
0,10023283211,1
1,10142254217,1
2,10165489824,1
3,10187291046,1
4,10192359162,1
...,...,...
4013,99938978285,1
4014,99974311662,1
4015,99976658299,1
4016,99987905597,1


In [51]:
merchants_pd["points"] = merchants_pd["points"] + merch_pop_mod_df["points"]
merchants_pd

,merchant_abn,points
0,10023283211,1.000
1,10142254217,1.000
2,10165489824,1.125
3,10187291046,1.125
4,10192359162,1.125
...,...,...
4013,99938978285,1.000
4014,99974311662,1.000
4015,99976658299,1.125
4016,99987905597,1.125


***

Get the merchant with the highest number of total transactions for each month (`ft_agg2`):

In [52]:
w = Window.partitionBy('month')

ft_agg2 = future_trans_sdf \
    .withColumn('maxT', F.max('total_transactions').over(w)) \
    .where(F.col('total_transactions') == F.col('maxT')) \
    .drop('maxT')

ft_agg2

year,month,merchant_abn,total_transactions
2023,1,24852446429,12706.867430622047
2023,2,24852446429,12725.976797500647
2023,3,24852446429,12601.879475370839
2023,4,24852446429,12624.618672374758
2023,5,24852446429,12557.475159081536
2023,6,24852446429,12576.861500222918
2023,7,24852446429,12540.356167994907
2023,8,24852446429,12555.113743323984
2023,9,24852446429,12535.352343858927
2023,10,24852446429,12546.091227819079


__[reference for above code](https://stackoverflow.com/questions/48829993/groupby-column-and-filter-rows-with-maximum-value-in-pyspark)__



For each month, order the merchants by increasing order of their total transactions <br>
(merchant with lowest number of transactions for that month will be at the top)

In [53]:
ordered_ft_pd = future_trans_sdf \
    .orderBy(asc("month"), asc("total_transactions")) \
    .drop("year") \
    .toPandas()

ordered_ft_pd

,month,merchant_abn,total_transactions
0,1,62346852889,21.097950
1,1,35235320738,21.149586
2,1,86791380020,21.154814
3,1,34455283435,21.199792
4,1,98082781436,21.204058
...,...,...,...
48211,12,46804135891,10163.244846
48212,12,49891706470,10724.271327
48213,12,64203420245,11311.885569
48214,12,86578477987,11805.480411


Note for below that `groupby` preserves ordering within each group

In [54]:
merch_abns = list(merchants_pd["merchant_abn"])

MONTHS = 12
for i in range(1, MONTHS+1):
    new_points = []
    
    month_data = ordered_ft_pd.groupby("month").get_group(i)
    month_data = pd.DataFrame(month_data.reset_index().drop(columns="index"))
    
    for abn in merch_abns:
        try:
            # the larger the index, the more transactions the merchant will have (ordered data)
            # thus more points awarded
            points = np.round(WEIGHT_TRANS*list(month_data.index[month_data["merchant_abn"] == abn])[0], 3)
        except IndexError:
            # have no data for the current abn
            points = 0

        new_points.append(points)

    merchants_pd["points"] = merchants_pd["points"] + new_points


In [55]:
merchants_pd

,merchant_abn,points
0,10023283211,14680.750
1,10142254217,14446.000
2,10165489824,184.125
3,10187291046,8435.625
4,10192359162,8808.000
...,...,...
4013,99938978285,17421.250
4014,99974311662,5447.875
4015,99976658299,17627.625
4016,99987905597,6729.750


***

Get the merchant with the highest profit for each month (`fp_agg2`):

In [56]:
w = Window.partitionBy('month')

fp_agg2 = future_profit_sdf \
    .withColumn('maxP', F.max('total_profit').over(w)) \
    .where(F.col('total_profit') == F.col('maxP')) \
    .drop('maxP')

fp_agg2

year,month,merchant_abn,total_profit
2023,1,32361057556,27740.570878728235
2023,2,32361057556,27878.7679430336
2023,3,32361057556,27552.75259841377
2023,4,32361057556,27653.557096675115
2023,5,32361057556,27479.095309639102
2023,6,32361057556,27551.766701347373
2023,7,32361057556,27461.515288951385
2023,8,32361057556,27514.536693437625
2023,9,32361057556,27471.280954645557
2023,10,32361057556,27511.178900430907


For each month, order the merchants by increasing order of their total profit <br>
(merchant with lowest total profit for that month at the top)

In [57]:
ordered_fp_pd = future_profit_sdf \
    .orderBy(asc("month"), asc("total_profit")) \
    .drop("year") \
    .toPandas()

ordered_fp_pd

,month,merchant_abn,total_profit
0,1,16156289887,169.090896
1,1,86791380020,177.765852
2,1,33068332703,179.699605
3,1,10441711491,179.786133
4,1,21350600012,179.972700
...,...,...,...
48211,12,96680767841,25475.481580
48212,12,45629217853,25860.368887
48213,12,86578477987,27096.311961
48214,12,48534649627,27268.520005


In [58]:
for i in range(1, MONTHS+1):
    new_points = []
    
    month_data = ordered_fp_pd.groupby("month").get_group(i)
    month_data = pd.DataFrame(month_data.reset_index().drop(columns="index"))
    
    for abn in merch_abns:
        try:
            # the larger the index, the more profit the merchant will have (ordered data)
            # thus more points awarded
            points = np.round(WEIGHT_PROFIT*list(month_data.index[month_data["merchant_abn"] == abn])[0], 3)
        except IndexError:
            # have no data for the current abn
            points = 0

        new_points.append(points)

    merchants_pd["points"] = merchants_pd["points"] + new_points

In [59]:
merchants_pd

,merchant_abn,points
0,10023283211,18909.250
1,10142254217,23638.000
2,10165489824,3668.250
3,10187291046,12815.250
4,10192359162,20447.250
...,...,...
4013,99938978285,31240.375
4014,99974311662,8866.000
4015,99976658299,35401.875
4016,99987905597,15740.250


***

Get the merchant with the highest number of middle customers for each month (`fmc_agg2`):

In [60]:
w = Window.partitionBy('month')

fmc_agg2 = future_mcustomers_sdf \
    .withColumn('maxMC', F.max('total_middle_customers').over(w)) \
    .where(F.col('total_middle_customers') == F.col('maxMC')) \
    .drop('maxMC')

fmc_agg2

year,month,merchant_abn,total_middle_customers
2023,1,24852446429,2796.8587999146635
2023,2,24852446429,2794.892202393302
2023,3,24852446429,2781.9611881292603
2023,4,24852446429,2781.828716128044
2023,5,24852446429,2775.997045578518
2023,6,24852446429,2776.425821608259
2023,7,24852446429,2773.9088483222054
2023,8,24852446429,2774.4608952929357
2023,9,24852446429,2773.4974495653432
2023,10,24852446429,2774.0415413334777


For each month, order the merchants by increasing order of their total predicted population of middle income earning customers <br>

In [61]:
ordered_fmc_pd = future_mcustomers_sdf \
    .orderBy(asc("month"), asc("total_middle_customers")) \
    .drop("year") \
    .toPandas()

ordered_fmc_pd

,month,merchant_abn,total_middle_customers
0,1,28836033916,4.850240
1,1,14626521979,4.850240
2,1,37670671283,4.850240
3,1,37706925794,4.850240
4,1,95594775419,4.850240
...,...,...,...
48211,12,46804135891,2379.631110
48212,12,49891706470,2465.908543
48213,12,64203420245,2576.013135
48214,12,86578477987,2670.840636


In [62]:
for i in range(1, MONTHS+1):
    new_points = []
    
    month_data = ordered_fmc_pd.groupby("month").get_group(i)
    month_data = pd.DataFrame(month_data.reset_index().drop(columns="index"))
    
    for abn in merch_abns:
        try:
            # the larger the index, the more profit the merchant will have (ordered data)
            # thus more points awarded
            points = np.round(WEIGHT_MCUSTOMERS*list(month_data.index[month_data["merchant_abn"] == abn])[0], 3)
        except IndexError:
            # have no data for the current abn
            points = 0

        new_points.append(points)

    merchants_pd["points"] = merchants_pd["points"] + new_points

In [63]:
merchants_pd

,merchant_abn,points
0,10023283211,23850.625
1,10142254217,28458.250
2,10165489824,3754.500
3,10187291046,15534.750
4,10192359162,23528.500
...,...,...
4013,99938978285,37048.125
4014,99974311662,10775.875
4015,99976658299,41278.625
4016,99987905597,17827.500


***

Get the top merchants

In [64]:
rankings = spark.createDataFrame(merchants_pd).sort(desc("points"))
rankings

merchant_abn,points
86578477987,42164.5
45629217853,42136.125
89726005175,42103.75
21439773999,42091.0
64403598239,42080.25
32361057556,42077.5
49891706470,42071.5
94493496784,42044.625
43186523025,42037.875
48534649627,42037.125


## Segmentation